In [ ]:
#Vincent
#1345375


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import matplotlib
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/mnist_train.csv', dtype=int)
X_train = train.drop('label', axis=1)
y_train = train['label']
test = pd.read_csv('../input/mnist_test.csv', dtype=int)
X_test = test.drop('label', axis=1)
y_test = test['label']

In [ ]:
max_depth=[10,20,30,40,50,60]
best_oob_score = 0
depthpoint = 0
best_ac = 0
best_odf = []
best_proba_1 = []
for d in range(len(max_depth)):
    model = ExtraTreesClassifier(bootstrap = True, oob_score=True,max_depth = max_depth[d],n_estimators=300, n_jobs=-1, random_state=1345375)
    model.fit(X_train,y_train)
    score = model.oob_score_
    preds = model.predict(X_test)
    ac = accuracy_score(y_test,preds)
    odf = model.oob_decision_function_
    proba = model.predict_proba(X_test) 
    print(str(score)+" "+str(max_depth[d])+" "+str(ac))
    if score > best_oob_score:
        best_oob_score = score
        depthpoint = max_depth[d]
        best_ac = ac
        best_odf = odf
        best_proba_1 = proba
print("Best score "+str(best_oob_score)+" Depth "+str(depthpoint)+" Accurancy_score " + str(best_ac))


In [ ]:
max_depth=[10,20,30,40,50,60]
best_oob_score_2 = 0
depthpoint_2 = 0
best_ac_2 = 0
best_odf_2 = []
best_proba_2 = []
def single_extra_tree(depth=None, seed=1345375):
    return ExtraTreesClassifier(max_depth=depth, n_estimators=1, bootstrap=False, random_state=seed)
for d in range(len(max_depth)):
    single_extra_tree
    boost = AdaBoostClassifier(base_estimator=single_extra_tree(depth=max_depth[d]), n_estimators=10, random_state=1345375)
    bag = BaggingClassifier(base_estimator=boost, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1345375)
    bag.fit(X_train,y_train)
    score = bag.oob_score_
    preds = bag.predict(X_test)
    ac = accuracy_score(y_test,preds)
    odf = bag.oob_decision_function_
    proba = bag.predict_proba(X_test)
    print(str(score)+" "+str(max_depth[d])+" "+str(ac))
    if score > best_oob_score_2:
        best_oob_score_2 = score
        depthpoint_2 = max_depth[d]
        best_ac_2 = ac
        best_odf_2 = odf
        best_proba_2 = proba
print("Best score "+str(best_oob_score_2)+" Depth "+str(depthpoint_2)+" Accurancy_score " + str(best_ac_2))

In [ ]:
num_components=[20, 40, 60]
best_oob_score_3 = 0
numberpoint_3 = 0
best_ac_3 = 0
best_odf_3 = []
best_proba_3 = []
for d in range(len(num_components)):
    pca = PCA(n_components=num_components[d], svd_solver='randomized', random_state=1345375)
    pip = Pipeline( [ ('pca', pca), ('ada_xt', AdaBoostClassifier(base_estimator=single_extra_tree(depth=depthpoint_2), n_estimators=10, random_state=1345375)) ])
    bag = BaggingClassifier(base_estimator=pip, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1345375)
    bag.fit(X_train,y_train)
    score = bag.oob_score_
    preds = bag.predict(X_test)
    ac = accuracy_score(y_test,preds)
    odf = bag.oob_decision_function_
    proba = bag.predict_proba(X_test)
    print(str(score)+" "+str(num_components[d])+" "+str(ac))
    if score > best_oob_score_3:
        best_oob_score_3 = score
        numberpoint_3 = num_components[d]
        best_ac_3 = ac
        best_odf_3 = odf
        best_proba_3 = proba
print("Best score "+str(best_oob_score_3)+" number "+str(numberpoint_3)+" Accurancy_score " + str(best_ac_3))

In [ ]:
sums1 = np.sum( [best_odf, best_odf_2, best_odf_3], axis=0)
accuracy_score(y_train,  np.argmax(sums1, axis=1))

In [ ]:
sums2 = np.sum( [best_odf, best_odf_2], axis=0)
accuracy_score(y_train,  np.argmax(sums2, axis=1))

In [ ]:
sums3 = np.sum( [best_odf_2, best_odf_3], axis=0)
accuracy_score(y_train,  np.argmax(sums3, axis=1))

In [ ]:
sums4 = np.sum( [best_odf, best_odf_3], axis=0)
accuracy_score(y_train,  np.argmax(sums4, axis=1))

In [ ]:
sums5 = np.sum( [best_odf], axis=0)
accuracy_score(y_train,  np.argmax(sums5, axis=1))

In [ ]:
sums6 = np.sum( [best_odf_2], axis=0)
accuracy_score(y_train,  np.argmax(sums6, axis=1))

In [ ]:
sums7 = np.sum( [best_odf_3], axis=0)
accuracy_score(y_train,  np.argmax(sums7, axis=1))

sums5 has the highest oob accuracy estimate<br>
which is only the ExtraTreesClassifier it self

In [ ]:
X_train_meta = np.concatenate( [best_odf, best_odf_2, best_odf_3], axis=1)
X_test_meta = np.concatenate( [best_proba_1, best_proba_2, best_proba_3], axis=1)

In [ ]:
log = LogisticRegression(random_state=1345375, C=50).fit(X_train_meta,y_train)
log_pred=log.predict(X_test_meta)
log_ac = accuracy_score(y_test,log_pred)
print(log_ac)

The accuracy is 0.9728<br>
Yes, it is better than simple voting, due to the reason that it has higher accuracy

In [ ]:
x=0
def misclassfied(n):
    item=n.values.reshape(28,28)
    plt.imshow(item, cmap = matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")
    plt.show()
    
for i in range(len(y_test)):
    if x == 11:
        break
    elif y_test[i]!=log_pred[i] and y_test[i] == x:
        print(y_test[i], "got wrong to : ", log_pred[i])
        misclassfied(X_test[i:i+1])
        x=x+1